In [3]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Pose model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Define angle function
def calculate_angle(a, b, c):
    """ Calculate angle between three points a, b, and c. """
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # Last point

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Open webcam
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize and convert color
        frame = cv2.resize(frame, (640, 480))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process with Mediapipe
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Extract required landmarks
            def get_landmark(name):
                return (landmarks[mp_pose.PoseLandmark[name].value].x,
                        landmarks[mp_pose.PoseLandmark[name].value].y)

            try:
                left_shoulder = get_landmark("LEFT_SHOULDER")
                right_shoulder = get_landmark("RIGHT_SHOULDER")
                left_hip = get_landmark("LEFT_HIP")
                right_hip = get_landmark("RIGHT_HIP")
                left_knee = get_landmark("LEFT_KNEE")
                right_knee = get_landmark("RIGHT_KNEE")

                # Check if body is fully visible (shoulders, hips, and knees detected)
                if all(left_shoulder) and all(right_shoulder) and all(left_hip) and all(right_hip) and all(left_knee) and all(right_knee):

                    # Compute back angle using shoulder-hip-knee
                    back_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                    # Display back angle
                    cv2.putText(frame, f"Back Angle: {int(back_angle)}°", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                    # Check for lean back error
                    if back_angle > 160:  # Threshold for excessive arching
                        cv2.putText(frame, "Error: Your back is arching too much!", (10, 60),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
                    else:
                        cv2.putText(frame, "✅ Correct Form", (10, 60),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error in detection: {e}")

            # Draw landmarks and skeleton
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3),
                                      connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2))

        # Show output
        cv2.imshow("Workout Form Analysis", frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
